In [1]:
import calendar
from datetime import datetime, timedelta
from typing import Dict, List

class Caregiver:
    def __init__(self, name: str, phone: str, email: str, is_paid: bool = False):
        self.name = name
        self.phone = phone
        self.email = email
        self.is_paid = is_paid
        self.pay_rate = 20.0 if is_paid else 0.0
        self.hours = 0
        self.availability = self._create_default_availability()
        
    def _create_default_availability(self) -> Dict:
        """Creates default weekly availability schedule"""
        schedule = {}
        for day in range(7):  # 0 = Monday, 6 = Sunday
            schedule[day] = {
                'AM': 'available',  # 7:00 AM - 1:00 PM
                'PM': 'available'   # 1:00 PM - 7:00 PM
            }
        return schedule
    
    def set_availability(self, day: int, shift: str, status: str):
        """Set availability for a specific day and shift"""
        if status not in ['preferred', 'available', 'unavailable']:
            raise ValueError("Status must be 'preferred', 'available', or 'unavailable'")
        self.availability[day][shift] = status
        
    def add_hours(self, hours: float):
        """Add worked hours"""
        self.hours += hours

class Schedule:
    def __init__(self, year: int, month: int):
        self.year = year
        self.month = month
        self.caregivers: List[Caregiver] = []
        self.schedule: Dict = {}
        
    def add_caregiver(self, caregiver: Caregiver):
        self.caregivers.append(caregiver)
        
    def generate_schedule(self):
        """Generate monthly schedule based on caregiver availability"""
        _, num_days = calendar.monthrange(self.year, self.month)
        
        for day in range(1, num_days + 1):
            weekday = calendar.weekday(self.year, self.month, day)
            
            # Find available caregivers for each shift
            am_shift = self._assign_shift(weekday, 'AM')
            pm_shift = self._assign_shift(weekday, 'PM')
            
            self.schedule[day] = {
                'AM': am_shift,
                'PM': pm_shift
            }
            
            # Add hours for paid caregivers
            if am_shift and am_shift.is_paid:
                am_shift.add_hours(6)
            if pm_shift and pm_shift.is_paid:
                pm_shift.add_hours(6)
    
    def _assign_shift(self, weekday: int, shift: str) -> Caregiver:
        """Assign a caregiver to a shift based on availability"""
        preferred = [c for c in self.caregivers if c.availability[weekday][shift] == 'preferred']
        available = [c for c in self.caregivers if c.availability[weekday][shift] == 'available']
        
        if preferred:
            return min(preferred, key=lambda x: x.hours)
        elif available:
            return min(available, key=lambda x: x.hours)
        return None

    def generate_html_calendar(self):
        """Generate HTML calendar with schedule"""
        html = f"""
        <html>
        <head>
            <title>Care Schedule - {calendar.month_name[self.month]} {self.year}</title>
            <style>
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid black; padding: 10px; text-align: center; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <h1>Care Schedule - {calendar.month_name[self.month]} {self.year}</h1>
            <table>
                <tr>
                    <th>Mon</th>
                    <th>Tue</th>
                    <th>Wed</th>
                    <th>Thu</th>
                    <th>Fri</th>
                    <th>Sat</th>
                    <th>Sun</th>
                </tr>
        """
        
        # Add calendar days
        first_day = calendar.weekday(self.year, self.month, 1)
        _, num_days = calendar.monthrange(self.year, self.month)
        
        current_day = 1
        html += "<tr>"
        
        # Add empty cells for days before the first of the month
        for _ in range(first_day):
            html += "<td></td>"
            
        # Add days of the month
        while current_day <= num_days:
            if (first_day + current_day - 1) % 7 == 0 and current_day != 1:
                html += "</tr><tr>"
                
            day_schedule = self.schedule.get(current_day, {'AM': None, 'PM': None})
            am_name = day_schedule['AM'].name if day_schedule['AM'] else "OPEN"
            pm_name = day_schedule['PM'].name if day_schedule['PM'] else "OPEN"
            
            html += f"""
                <td>
                    {current_day}<br>
                    <b>AM:</b> {am_name}<br>
                    <b>PM:</b> {pm_name}
                </td>
            """
            
            current_day += 1
            
        # Add empty cells for remaining days in the last week
        while (first_day + current_day - 1) % 7 != 0:
            html += "<td></td>"
            current_day += 1
            
        html += """
                </tr>
            </table>
        </body>
        </html>
        """
        
        with open(f"care_schedule_{self.year}_{self.month}.html", "w") as f:
            f.write(html)

In [2]:
def generate_pay_report(schedule: Schedule):
    """Generate weekly pay report for paid caregivers"""
    report = f"Pay Report for {calendar.month_name[schedule.month]} {schedule.year}\n"
    report += "=" * 50 + "\n\n"
    
    paid_caregivers = [c for c in schedule.caregivers if c.is_paid]
    total_monthly_pay = 0
    
    for caregiver in paid_caregivers:
        weekly_hours = caregiver.hours / 4  # Approximate weekly hours
        weekly_pay = weekly_hours * caregiver.pay_rate
        monthly_pay = weekly_pay * 4
        
        report += f"Caregiver: {caregiver.name}\n"
        report += f"Weekly Hours: {weekly_hours:.2f}\n"
        report += f"Weekly Pay: ${weekly_pay:.2f}\n"
        report += f"Monthly Pay: ${monthly_pay:.2f}\n"
        report += "-" * 30 + "\n"
        
        total_monthly_pay += monthly_pay
    
    report += f"\nTotal Monthly Payroll: ${total_monthly_pay:.2f}"
    
    with open(f"pay_report_{schedule.month}_{schedule.year}.txt", "w") as f:
        f.write(report)

In [3]:
# Create schedule for current month
current_date = datetime.now()
schedule = Schedule(current_date.year, current_date.month)

# Add caregivers
caregivers = [
    Caregiver("Alice Smith", "555-0101", "alice@email.com", True),
    Caregiver("Bob Jones", "555-0102", "bob@email.com", True),
    Caregiver("Carol Wilson", "555-0103", "carol@email.com", False),
    Caregiver("David Brown", "555-0104", "david@email.com", True),
    Caregiver("Eve Davis", "555-0105", "eve@email.com", False),
    Caregiver("Frank Miller", "555-0106", "frank@email.com", True),
    Caregiver("Grace Taylor", "555-0107", "grace@email.com", False),
    Caregiver("Henry White", "555-0108", "henry@email.com", True)
]

for caregiver in caregivers:
    schedule.add_caregiver(caregiver)

# Set some availability preferences (example)
caregivers[0].set_availability(5, 'AM', 'unavailable')  # Alice unavailable Saturday mornings
caregivers[1].set_availability(0, 'AM', 'preferred')    # Bob prefers Monday mornings

# Generate schedule
schedule.generate_schedule()

# Generate HTML calendar
schedule.generate_html_calendar()

# Generate pay report
generate_pay_report(schedule)